You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

# Progetto di _Foundations of Computer Science_

Camagni Valentina (878252), Grosso Silvia (881993), Merelli Elisa (881427)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#LIBRERIE:
import pandas as pd
import re
import numpy as np

Importo i dati:
usiamo l'encoding richiesto 'latin-1', e la modalità di compression = 'gzip'; inoltre impostiamo il parametro low_memory = False, come segnalato da un Warning. I primi tre file sono sono separati da ';', mentre gli ultimi due da ',', infine usiamo *dtype* per verificare il tipo del contentuo dei campi.

In [3]:
books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz", compression = 'gzip', sep =';', escapechar = '\\', encoding = 'latin-1', low_memory=False, dtype = {'Year-Of-Publication':'int64'})

In [4]:
#aggiungendo l'escape character '\\' in books non ho errore in questo record
books.iloc[209538]

ISBN                                                          078946697X
Book-Title             DK Readers: Creating the X-Men, How It All Beg...
Book-Author                                           Michael Teitelbaum
Year-Of-Publication                                                 2000
Publisher                                              DK Publishing Inc
Image-URL-S            http://images.amazon.com/images/P/078946697X.0...
Image-URL-M            http://images.amazon.com/images/P/078946697X.0...
Image-URL-L            http://images.amazon.com/images/P/078946697X.0...
Name: 209538, dtype: object

In [ ]:
bookrat = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz", compression = 'gzip', sep =';', encoding = 'latin-1', dtype = {'User-ID':'int64',
                              'Book-Rating':'int64'})

In [ ]:
users = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz", compression = 'gzip', sep =';', encoding = 'latin-1', dtype = {'User-ID': 'int64',
                            'Age':'float64'})

In [ ]:
gb_books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz", compression = 'gzip', sep =',', encoding = 'latin-1', 
                       dtype = {range(1,5) :'int64', 
                              'original_publication_year':'float64',
                              'average_rating':'float64',
                              'ratings_count':'int64', 
                              'work_ratings_count':'int64', 
                              'work_text_reviews_count':'int64',
                              'ratings_1':'int64',
                              'ratings_2':'int64', 
                              'ratings_3':'int64',
                              'ratings_4':'int64', 
                              'ratings_5':'int64'})

In [ ]:
gb_rat = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz", compression = 'gzip', sep =',', encoding = 'latin-1', dtype = {range(1,3):'int64'})

Esploriamo i dati appena caricati:

In [ ]:
books.shape

In [ ]:
books.head(50)

In [ ]:
bookrat.shape

In [ ]:
bookrat.head(10)

In [ ]:
users.shape

In [ ]:
users.head(10)

In [ ]:
gb_books.shape

In [ ]:
gb_books.head(10)

In [ ]:
gb_books.columns

In [ ]:
gb_rat.shape

In [ ]:
gb_rat.head(10)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

Abbiamo osservato che in *'Location'* le stringhe sono separate da virgola e spazio. Utilizzo il metodo split specificando l'espressione regolare di separazione in pat:

In [ ]:
split=users['Location'].str.split(',\s', expand=True)
split[split[8].isna()==False]

Notiamo che di default vengono creati 9 colonne poichè, evidentemente, vi sono degli users con più campi relativamente alla *'Location'*. Decidiamo di definire attraverso una regex il formato *'Location'* che desideriamo. Gli users che non soddisfano tale pattern non verranno considerati:

In [ ]:
users=users[users['Location'].str.match("^([a-zA-Z\.\s\/-]+|""),([a-zA-Z\.\s\/-]+|""),([a-zA-Z\.\s\/-]+|"")$")==True].reset_index(drop=True)
users.head()

In [ ]:
users.shape

Splittiamo la colonna Location nei 3 campi *'City'*, *'Region'*, *'Country'*, richiesti

In [ ]:
users[['City','Region','Country']]=users['Location'].str.split(',\s', expand=True)
users.head()

Dopodiché definiamo un nuovo dataframe *users_normalize* dove rimuoviamo la colonna *'Location'*:

In [ ]:
users_normalize = users.drop('Location', 1)
users_normalize.head()

### 2. For each book in the *Books* dataset, compute its average rating.

Osservando i dataset, si vede che il dataset *book_rat* vi son più righe riferite allo stesso ISBN in quanto riportano valutazioni di Users differenti. Dunque raggruppiamo per ISBN e calcoliamo la media del 'rating'.

In [ ]:
bookrat.groupby('ISBN').mean()['Book-Rating']

Verifichiamo sul secondo ISBN: qui la media dovrebbe essere 18/7=2.57, e non torna con il valore indicato in Book-Rating. Individuiamo le righe conteneti la stringa '0375404120' in ISBN con il metodo *contains*:

In [ ]:
bookrat[bookrat['ISBN']=='0375404120']

In [ ]:
bookrat[bookrat['ISBN'].str.contains('0375404120')]

Notiamo che vi sono più righe che contengono la stringa '0375404120' rispetto alla ricerca precendente dunque deduciamo che la presenza di spazi all'inizio e alla fine della stringa hanno introdotto errori nel raggruppamento:
infatti verifichiamo che inserendo uno spazio all'inizio della stringa torna il risultato mostrato nella colonna Book-Rating iniziale:

In [ ]:
bookrat[bookrat['ISBN'].str.contains(' 0375404120')] 

quindi decidiamo di togliere tutti gli spazi a capo e coda delle stringhe dalla colonna ISBN attraverso il metodo strip, nei dataframe *books* e *bookrat*:

In [ ]:
bookrat.ISBN = bookrat.ISBN.str.strip()

In [ ]:
books.ISBN = books.ISBN.str.strip()

ora possiamo applicare il group by per dividere in base all'ISBN:

In [ ]:
#specifichiamo as_index = False per non avere ISBN come indice

bookrat_avg = bookrat.groupby('ISBN', as_index = False).mean()[['ISBN','Book-Rating']]
bookrat_avg.head()

Ora possiamo fare il merge dei due dataframe sull'ISBN:

In [ ]:
books_bookrat_avg = pd.merge(books, bookrat_avg, how = 'left', on = 'ISBN')

In [ ]:
books_bookrat_avg['Book-Rating']=books_bookrat_avg['Book-Rating'].round(2)

In [ ]:
books_bookrat_avg[['ISBN','Book-Title','Book-Author','Book-Rating']].head(10)

### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [ ]:
gb_books.columns

Si osserva che è presente la colonna contenente le valutazioni medie per ogni libro:

In [ ]:
gb_books[['book_id','isbn','original_title','average_rating']].set_index('book_id')

Controlliamo che i valori contenuti nella colonna *'average_rating'* corrispondano alla media pesata del numero di persone che hanno votato da 1 a 5 (contenuto nelle colonne *'ratings_1-5'*) sul numero totale di persone che hanno votato (contenuto in *'work_ratings_count'*):

In [ ]:
#verifichiamo sia vero (quante uguaglianze false) che per ogni riga che la somma dei 5 valori di ratings_1-5 dia 'work_ratings_count':
gb_books[(gb_books['work_ratings_count'] == gb_books['ratings_1'] + gb_books['ratings_2'] + gb_books['ratings_3'] + gb_books['ratings_4'] + gb_books['ratings_5']) == False].count()

In [ ]:
#verifichiamo ora la corrispondenza dei valori delle medie:
gb_books[(gb_books['average_rating'] == ((gb_books['ratings_1']*1 + gb_books['ratings_2']*2 + gb_books['ratings_3']*3 + gb_books['ratings_4']*4 + gb_books['ratings_5']*5)/(gb_books['work_ratings_count'])).round(2)) == False].count()

### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

Come prima cosa creiamo *merge* in cui raggruppiamo con i tre attributi *'Book-Title'*, *'Book-Author'* e *'Publisher'*, riportando il conteggio delle ricorrenze.

In [ ]:
# as_index = False per fare si che le colonne 'Book-Title', 'Book-Author', 'Publisher' rimangano tali e non diventino indice:
merge = books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index = False).count()

Ora consideraimo solo i dati che appaiono più di una volta (come having di SQL):

In [ ]:
merged_books = merge[merge['ISBN']>1]

In [ ]:
merged_books = merged_books[['Book-Title', 'Book-Author', 'Publisher','ISBN']].rename({'ISBN':'count'}, axis = 1)

In [ ]:
merged_books.head(10)

In [ ]:
len(merged_books)
# I libri con stesso titolo, autore e editore sono 4725.

### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

Unisco le tabelle *books* e *bookrat* con ISBN chiave primaria di *books* (già "puliti" precedentemente), in modo da avere per ogni libro in *book* la votazione data da ogni User che lo ha valutato:

In [ ]:
books_bookrat = pd.merge(books, bookrat, how = 'left',on = 'ISBN')
books_bookrat.head()

Facciamo una left join tra *merged_books* e *books_bookrat*, con *'Book-Title'*, *'Book-Author'* e *'Publisher'*:

In [ ]:
merged_books_bookrat = pd.merge(merged_books[['Book-Title', 'Book-Author', 'Publisher']],books_bookrat , how = 'left', on = ['Book-Title', 'Book-Author', 'Publisher'])

In [ ]:
merged_books_bookrat.head()

In [ ]:
len(merged_books_bookrat)
# Otteniamo un numero di records maggiore: infatti, lo stesso libro individuato da 'Book-Title', 'Book-Author' e 
# 'Publisher' può avere edizioni e quindi ISBN differenti, e per ogni edizione, valutazioni da utenti differenti.

Per ogni libro in *merged_books*, calcoliamo la media delle valutazioni degli utenti, raggruppando anche per *'ISBN'*, ottenendo così la media per ogni edizione:

In [ ]:
merged_books_avg = merged_books_bookrat.groupby(['Book-Title', 'Book-Author', 'Publisher', 'ISBN'], as_index = False).mean('Book-Rating').round(2)[['Book-Title', 'Book-Author', 'Publisher','Book-Rating','ISBN']].rename({'Book-Rating':'Avg-Rating'}, axis = 1)

Abbiamo ottenuto un numero maggiore del numero di libri in *merged_books*, questo perchè in questa tabella possiamo sapere, per ogni libro che ha avuto più edizioni, la valutazione media di ogni singola edizione

In [ ]:
merged_books_avg.head(20)

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

*merged_books_avg* è il dataframe che contiene per ogni libro di *merged_books* (definito da Title, Author, Publisher) la media dei ratings su ogni edizione (ISBN). E', quindi, sufficiente calcolare il minimo e il massimo avg_rating raggruppando per i tre attributi.

In [ ]:
dfmin = merged_books_avg.groupby(['Book-Title','Book-Author','Publisher'], as_index = False).min()[['Book-Title','Book-Author','Publisher','Avg-Rating']].rename({'Avg-Rating':'Min-Rating'}, axis=1)

In [ ]:
seriemax = merged_books_avg.groupby(['Book-Title','Book-Author','Publisher'], as_index = False).max()['Avg-Rating']

In [ ]:
pd.concat([dfmin,seriemax], axis = 1).reset_index(drop= True).rename({'Avg-Rating':'Max-Rating'}, axis=1)

In [ ]:
# Osserviamo che come ci si aspetterebbe, otteniamo nuovamente 4725 righe come in merged_books.

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [ ]:
gb_books.head()

Creo una lista con gli autori:

In [ ]:
lista_autori = gb_books.authors.str.split(', ').tolist() 
lista_autori

Creiamo un dizionario *libri_autori* che abbia come chiavi i valori contenuti nella colonna *'book_id'* di *gb_books*, e come valore una lista contenete gli autori del libro:

In [ ]:
libri_autori={}
for i in list(gb_books['book_id']):
    libri_autori[i] = lista_autori[i-1]
                                        # il -1 gestisce il fatto che l'indice implicito di gb_books parta
                                        # da zero, mentre book_id parta da uno.
libri_autori

La seconda parte dell'esercizio richiede di calcolare per ogni autore lo *shared number of reviews with a text*:

In [ ]:
#creo una lista degli autori come somma delle liste presenti in lista_autori, senza duplicati(set) e ordinati (sorted):
singoli_autori = sorted(set(sum(lista_autori,[])))
singoli_autori

Creo la serie *rev* come copia di *'work_text_reviews_count'* e poi ad ogni elemento sostituisco la porzione di *shared number of reviews with a text* che andrà assegnata a ciascun coautore del libro, ossia divido il valore di *'work_text_reviews_count'* per il numero di autori che hanno scritto il libro.

In [ ]:
rev = gb_books['work_text_reviews_count'].copy()
for i in range(len(rev)):
    rev[i] = rev[i]/len(libri_autori[i+1])
rev  

Creo un dizionario che associa a ciascun autore lo *'shared number of reviews with a text'*:

In [ ]:
shar_num_rev = {}
for autore in singoli_autori:
    shar_num_rev[autore]=0
    for i in libri_autori:
        if autore in libri_autori[i]:
            shar_num_rev[autore]+=rev[i-1]

In [ ]:
shar_num_rev

### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [ ]:
#Creiamo un insieme di anni in cui sono stati pubblicati libri:
years = set(gb_books['original_publication_year'])

Creiamo un dizionario che ha come chiavi gli anni di pubblicazione (*'original_publication_year'*) e come valori la lista degli autori dei libri che hanno pubblicato in quell'anno.

In [ ]:
year_authors = {}
for year in list(years):
    year_authors[year] = []
    for i in gb_books.book_id:
        if gb_books.original_publication_year[i-1] == year:
            year_authors[year]+=gb_books.authors[i-1].split(', ')
year_authors

Creiamo un nuovo dizionario in cui le chiavi sono gli anni di pubblicazione e i valori sono gli autori che hanno pubblicato in quell'anno con maggiore numero di *shared number of reviews with a text*; in caso di parità vengono stampati tutti gli autori con tale valore.

In [ ]:
anno_max_rev={}
for year in years:
    massimo=0
    anno_max_rev[year]=[]
    for autore in year_authors[year]:
        if shar_num_rev[autore]>=massimo:
            massimo=shar_num_rev[autore]
            anno_max_rev[year]+=[autore]
            
anno_max_rev

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

Definiamo *diff_avg* il merge tra i dataset *books_bookrat_avg* (contenente per ogni libro in *books* il rating medio) e *gb_books* (dove per ogni libro vi è già l'average rating in *'Average-Rating'*).
La differenza dell'average rating è calcolata dalla semplice differenza delle due colonne per ogni libro.

In [ ]:
diff_avg = pd.merge(books_bookrat_avg,gb_books, how = 'inner', left_on='ISBN', right_on='isbn')

In [ ]:
diff_avg['diff_avg'] = diff_avg['Book-Rating']-diff_avg['average_rating']

In [ ]:
diff_avg[['ISBN','Book-Title','Year-Of-Publication','Publisher','Book-Rating','average_rating','diff_avg']]

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

Creiamo innanzitutto il DataFrame contenente gli users con età sconosciuta:

In [ ]:
df_unknown = users[users['Age'].isna()].reset_index(drop=True)
df_unknown

Creiamo la lista con i ranges d'età:

In [ ]:
ranges= [-1]+[i for i in range(14,int(users.Age.max())+1,10)]
ranges

Creiamo il dizionario le cui chiavi sono i nomi dei DataFrame distinti per età, e i valori sono i DataFrame contenenti l'insieme dei record degli users filtrati per età associata:

In [ ]:
classi_eta = {}
for i in range(0,len(ranges)-1):
    classi_eta["df_{0}".format(ranges[i+1])] = users[(users['Age']>ranges[i])&(users['Age']<=ranges[i+1])].sort_values('Age').reset_index(drop=True)

In [ ]:
classi_eta

In [ ]:
classi_eta['df_14']

In [ ]:
classi_eta['df_244']

### 11. Find the books that appear only in the goodbooks datasets.

Stampiamo i libri con ISBN in *good_books* che non sia in *books* (eseguiamo l'equivalente pandas del comando NOT IN di SQL).

In [ ]:
gb_books[~ gb_books['isbn'].isin(books['ISBN'])]

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

Raggruppiamo in *times* i record di *books* secondo la coppia (*'Book-Title'*, *'Book-Author'*) e contiamo la numerosità di ogni gruppo.

In [ ]:
times = books.groupby(['Book-Title','Book-Author'], as_index = False).count()[['Book-Title','Book-Author','ISBN']].rename({'ISBN':'count'}, axis = 1)
times

Mostriamo la coppia (*'Book-Title'*, *'Book-Author'*) con il conteggio massimo:

In [ ]:
times[times['count'] == times['count'].max()]

### 13. Find the author with the highest average rating according to the goodbooks datasets.

Creaimo un dizionario che associa ad ogni autore tutti i *book_id* in cui è presente.

In [ ]:
autori_libri = {}
for autore in singoli_autori:
    autori_libri[autore]=[]
    for book_id in libri_autori:
        if autore in libri_autori[book_id]:  
            autori_libri[autore]+=[book_id]
autori_libri

Creiamo un dizionario che ad ogni autore associa il valore di *average_rating* dei libri in cui è presente.

In [ ]:
high_avg_au = {}
for autore in singoli_autori:
    high_avg_au[autore]=[]
    for i in autori_libri[autore]:
        high_avg_au[autore]+=[gb_books.average_rating[i-1]]
    
high_avg_au

Associamo ad ogni autore il valore medio degli *average_rating* associati.

In [ ]:
avg_rating = {}
for autore in high_avg_au:
    avg_rating[autore] = np.mean(high_avg_au[autore]).round(2)
avg_rating

Individuiamo l'autore con massimo valore medio di *average_rating*.

In [ ]:
max(avg_rating, key = avg_rating.get), max(avg_rating.values())   